In [45]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from datetime import datetime
# https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/discussion/31067

In [107]:
data_path = "../input/"
test_X = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')
train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))
sub_id = test_X.listing_id.values

In [95]:
test = pd.read_csv('../blend/train_blend_LightGBM_BM_0331_2017-04-01-07-33.csv',header=None)

In [97]:
test_1 = test.loc[:,[0,1,2]]
test_1.columns = ['low', 'medium', 'high']

In [98]:
train_y.shape

(49352L,)

In [99]:
print log_loss(train_y,test_1.iloc[:,:3])

0.517783695959


In [56]:
interest_levels = ['low', 'medium', 'high']

tau = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df):
    y = df[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    return df_correct

In [100]:
test_correct = correct(test_1)

[0.98814302630854334, 1.0216313493613942, 1.0463695819468612]
[0.9944797334284845, 1.0093311148887807, 1.0226538178600511]


In [101]:
print log_loss(train_y,test_correct)

0.517590767658


In [109]:
sub_test = pd.read_csv('../blend/test_blend_LightGBM_mean_BM_0331_2017-04-01-07-33.csv',header=None)

sub_test = sub_test.loc[:,[0,1,2]]
sub_test.shape  

sub_test.columns = ["low", "medium", "high"]
sub_test = correct(sub_test)

[0.98958039627628713, 1.0082966553717279, 1.0752354439484375]
[0.994441687176286, 1.0079542495170024, 1.0272869687764119]


In [112]:
out_df.head()

,low,medium,high,listing_id
0,0.359409,0.579358,0.061233,7142618
1,0.968987,0.020265,0.010748,7210040
2,0.920736,0.076147,0.003117,7174566
3,0.170297,0.561318,0.268385,7191391
4,0.825239,0.166347,0.008414,7171695


In [110]:
sub_name = '../output/sub_LightGBM_BM_0331_2017-04-01-07-33_53396_correct.csv'

out_df = sub_test.copy()

out_df["listing_id"] = sub_id
out_df.to_csv(sub_name, index=False)

In [115]:
def correct1(df,tau1):
    y = df[interest_levels].mean()
    a = [tau1[k] / y[k]  for k in interest_levels]
#     print a

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau1[k] / y[k]  for k in interest_levels]
#     print a

    return df_correct
def myfunc(x):
    tau1 = {
    'low': x[0], 
    'medium': x[1],
    'high': 1-x[0]-x[1], 
    }
    tmp = log_loss(train_y,correct1(test_1,tau1))
    print x,tmp
    return tmp

In [116]:
from scipy.optimize import minimize

In [117]:
c = np.array([0.69195995,0.23108864])
logloss = minimize(myfunc,c,method='Nelder-Mead', tol=1e-6)

[ 0.7  0.2] 0.521042220195
[ 0.735  0.2  ] 0.520507102337
[ 0.7   0.21] 0.518796139109
[ 0.735  0.21 ] 0.521775933402
[ 0.70875  0.2025 ] 0.519546115925
[ 0.67375  0.2125 ] 0.521784214582
[ 0.7196875  0.203125 ] 0.519150036399
[ 0.7109375  0.210625 ] 0.518356717145
[ 0.71203125  0.2146875 ] 0.518242036955
[ 0.69234375  0.2215625 ] 0.517806925339
[ 0.67867187  0.23078125] 0.517877968346
[ 0.704375  0.22625 ] 0.518149729444
[ 0.6846875  0.233125 ] 0.517544080599
[ 0.67101562  0.24234375] 0.517916630683
[ 0.67265625  0.2284375 ] 0.518694778195
[ 0.69644531  0.22679688] 0.517615103507
[ 0.68878906  0.23835938] 0.517963573575
[ 0.69145508  0.22576172] 0.517583751077
[ 0.67969727  0.23208984] 0.517734060981
[ 0.6922583   0.22812012] 0.517538036087
[ 0.68549072  0.2354834 ] 0.517593240876
[ 0.68996399  0.22819214] 0.517536671934
[ 0.69753479  0.22318726] 0.517634228817
[ 0.68789932  0.23064056] 0.517524160144
[ 0.68560501  0.23071259] 0.517554553814
[ 0.69059498  0.22876823] 0.517527675296
[ 

In [135]:
tau_1 = {
    'low': logloss.final_simplex[0].mean(axis =0)[0], 
    'medium': logloss.final_simplex[0].mean(axis =0)[1],
    'high': 1-logloss.final_simplex[0].mean(axis =0)[0]-logloss.final_simplex[0].mean(axis =0)[1], 
}

In [136]:
tau_1

{'high': 0.080864613188141504,
 'low': 0.68879748644126548,
 'medium': 0.23033790037059301}

In [134]:
logloss.final_simplex[0].mean(axis =0)[1]

0.23033790037059301

In [137]:
sub_test = pd.read_csv('../blend/test_blend_LightGBM_mean_BM_0331_2017-04-01-07-33.csv',header=None)

sub_test = sub_test.loc[:,[0,1,2]]
sub_test.shape  

sub_test.columns = ["low", "medium", "high"]
sub_test = correct1(sub_test,tau_1)


sub_name = '../output/sub_LightGBM_BM_0331_2017-04-01-07-33_53396_correct_NM.csv'

out_df = sub_test.copy()

out_df["listing_id"] = sub_id
out_df.to_csv(sub_name, index=False)

In [25]:
sub_name = '../output/sub_test_correct_2nd_2017-03-28-20-37_1' + '.csv'

out_df = pd.DataFrame(test_correct)
out_df.to_csv(sub_name, index=False)

In [26]:
test_correct.head(10)

,low,medium,high,listing_id
0,0.379459,0.539948,0.080594,7142618
1,0.994223,0.004460,0.001317,7210040
2,0.960689,0.036720,0.002591,7174566
3,0.198045,0.543916,0.258039,7191391
4,0.872846,0.122745,0.004410,7171695
5,0.881218,0.116865,0.001918,7225206
6,0.993222,0.005570,0.001209,7200075
7,0.195406,0.546552,0.258042,7145074
8,0.949287,0.045938,0.004775,7193645
9,0.997492,0.002221,0.000287,7147703


In [27]:
test.head(10)

,low,medium,high,listing_id
0,0.362648,0.548257,0.089095,7142618
1,0.993740,0.004736,0.001523,7210040
2,0.958103,0.038908,0.002989,7174566
3,0.184328,0.537863,0.277808,7191391
4,0.865611,0.129330,0.005059,7171695
5,0.874571,0.123227,0.002201,7225206
6,0.992688,0.005914,0.001397,7200075
7,0.181844,0.540386,0.277770,7145074
8,0.945865,0.048631,0.005504,7193645
9,0.997309,0.002359,0.000332,7147703
